# Dataiku Takehome assignment - Part 1 - Data Cleansing

## Background Information
The United States Census Bureau leads the country’s Federal Statistical System; its primary
responsibility is to collect demographic and economic data about America to help inform
strategic initiatives. Every ten years, the census is conducted to collect and organize information
regarding the US population with the intention of effectively allocating billions of dollars of
funding to various endeavors (e.g., the building and maintenance of hospitals, schools, fire
departments, transportation infrastructure, etc.). Additionally, the collection of census
information helps to examine the demographic characteristics of subpopulations across the
country.

## The Data
You have been provided a sample dataset from the US Census archive containing detailed, but
anonymized, information for ~300,000 individuals. This archive contains four files:
1. `census_income_learn.csv`
2. `census_income_test.csv`
3. `census_income_metadata.txt`
4. `census_income_additional_info.pdf (supplemental information)`

## The Task
For this technical assessment, you have been tasked with identifying **characteristics that are
associated with a person making more or less than $50,000 per year**; the target variable for
your research question is the final column of the datasets.


As the data scientist on this project, you are to attempt to answer this question by constructing a
data analysis/modeling pipeline. Code submissions should be in Python and making the solution
easily readable and replicable by the team will give you additional marks. In the event you would
like to use a different language or tool, please ask. Considerations for your data analysis should
include, but are not limited to, the following:
- **Data Preparation: Data cleaning, preprocessing, feature engineering, etc., that may aid
in improving data clarity & model generation.**
- Exploratory Data Analysis: Numerical and/or graphical representations of the data that
may help inform insights and/or tactics for answering the research question of interest.
- Data Modeling: The building of a few competing models to predict the target variable.
- Model Assessment: A selection of the best model based on performance comparisons.
- Results: A concise summary of key findings, recommendations, & future improvements.

In [1]:
!cat ./data-raw/census_income_metadata.txt

| This data was extracted from the census bureau database found at
| https://www.census.gov/data.html
| Donor: Terran Lane and Ronny Kohavi
|        Data Mining and Visualization
|        Silicon Graphics.
|        e-mail: terran@ecn.purdue.edu, ronnyk@sgi.com for questions.
|
| The data was split into train/test in approximately 2/3, 1/3
| proportions using MineSet's MIndUtil mineset-to-mlc.
|
| Prediction task is to determine the income level for the person
| represented by the record.  Incomes have been binned at the $50K
| level to present a binary classification problem, much like the
| original UCI/ADULT database.  The goal field of this data, however,
| was drawn from the "total person income" field rather than the
| "adjusted gross income" and may, therefore, behave differently than the
| original ADULT goal field.
|
| More information detailing the meaning of the attributes can be
| found in http://www.bls.census.gov/cps/cpsmain.htm
| To make use of the data descriptions at th

# Data Preparation

**Tasks**
1. Column names mapping (Found column `AGI`, `FEDTAX`, `PEARNVAL`, `PTOTVAL`, `TAXINC` could not match with data)
2. Data validity check

In [2]:
import pandas as pd
import seaborn as sns

In [3]:
colnames = [
    "AAGE",        # age
    "ACLSWKR",     # class of worker
    "ADTIND",      # industry code
    "ADTOCC",      # occupation code
    # "AGI",         # adjusted gross income - Not in data
    "AHGA",        # education
    "AHRSPAY",     # wage per hour
    "AHSCOL",      # enrolled in edu inst last wk
    "AMARITL",     # marital status
    "AMJIND",      # major industry code
    "AMJOCC",      # major occupation code
    "ARACE",       # mace (Race?)
    "AREORGN",     # hispanic Origin
    "ASEX",        # Sex
    "AUNMEM",      # member of a labor union
    "AUNTYPE",     # reason for unemployment
    "AWKSTAT",     # full or part time employment stat
    "CAPGAIN",     # capital gains
    "CAPLOSS",     # capital losses
    "DIVVAL",      # divdends from stocks
    # "FEDTAX",      # federal income tax liability - Not in data
    "FILESTAT",    # tax filer status
    "GRINREG",     # region of previous residence
    "GRINST",      # state of previous residence
    "HHDFMX",      # detailed household and family stat
    "HHDREL",      # detailed household summary in household
    "MARSUPWT",    # instance weight
    "MIGMTR1",     # migration code-change in msa
    "MIGMTR3",     # migration code-change in reg
    "MIGMTR4",     # migration code-move within reg
    "MIGSAME",     # live in this house 1 year ago
    "MIGSUN",      # migration prev res in sunbelt
    "NOEMP",       # num persons worked for employer
    "PARENT",      # family members under 18
    # "PEARNVAL",    # total person earnings - Not in data
    "PEFNTVTY",    # country of birth father
    "PEMNTVTY",    # country of birth mother
    "PENATVTY",    # country of birth self
    "PRCITSHP",    # citizenship
    # "PTOTVAL",     # total person income - Not in data
    "SEOTR",       # own business or self employed
    # "TAXINC",      # taxable income amount - Not in data
    "VETQVA",      # fill inc questionnaire for veteran's admin
    "VETYN",       # veterans benefits
    "WKSWORK",     # weeks worked in year
    "year",        # Year - 94 or 95
    "target"       # Target Variable
]

In [4]:
train_data = pd.read_csv("./data-raw/census_income_learn.csv", header=None, names=colnames)
test_data = pd.read_csv("./data-raw/census_income_test.csv", header=None, names=colnames)

In [5]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [6]:
train_data.shape

(199523, 42)

In [7]:
test_data.shape

(99762, 42)

### Clean data
- Remove heading and trailing spaces by `strip()`
- Check if there are mismatched values
- Knowing for unknown values, they will be represented by `?`. I am not changing it to anything else. 

In [9]:
train_data_categorical = train_data.select_dtypes(include=['object']).apply(lambda x: x.str.strip(), axis = 1)
test_data_categorical = test_data.select_dtypes(include=['object']).apply(lambda x: x.str.strip(), axis = 1)

train_data.loc[:, train_data_categorical.columns] = train_data_categorical
test_data.loc[:, test_data_categorical.columns] = test_data_categorical

In [10]:
EXPECTED_UNIQUE_VALUES = {
    "ACLSWKR": ["Not in universe",
              "Federal government",
              "Local government",
              "Never worked",
              "Private",
              "Self-employed-incorporated",
              "Self-employed-not incorporated",
              "State government",
              "Without pay"],
    "AHGA": ["Children",
           "7th and 8th grade",
           "9th grade",
           "10th grade",
           "High school graduate",
           "11th grade",
           "12th grade no diploma",
           "5th or 6th grade",
           "Less than 1st grade",
           "Bachelors degree(BA AB BS)",
           "1st 2nd 3rd or 4th grade",
           "Some college but no degree",
           "Masters degree(MA MS MEng MEd MSW MBA)",
           "Associates degree-occup /vocational",
           "Associates degree-academic program",
           "Doctorate degree(PhD EdD)",
           "Prof school degree (MD DDS DVM LLB JD)"],
    "AHSCOL": ["Not in universe", "High school", "College or university"],
    "AMARITL": ["Never married",
              "Married-civilian spouse present",
              "Married-spouse absent",
              "Separated",
              "Divorced",
              "Widowed",
              "Married-A F spouse present"],
    "AMJIND": ["Not in universe or children",
             "Entertainment",
             "Social services",
             "Agriculture",
             "Education",
             "Public administration",
             "Manufacturing-durable goods",
             "Manufacturing-nondurable goods",
             "Wholesale trade",
             "Retail trade",
             "Finance insurance and real estate",
             "Private household services",
             "Business and repair services",
             "Personal services except private HH",
             "Construction",
             "Medical except hospital",
             "Other professional services",
             "Transportation",
             "Utilities and sanitary services",
             "Mining",
             "Communications",
             "Hospital services",
             "Forestry and fisheries",
             "Armed Forces"],
    "AMJOCC": ["Not in universe",
             "Professional specialty",
             "Other service",
             "Farming forestry and fishing",
             "Sales",
             "Adm support including clerical",
             "Protective services",
             "Handlers equip cleaners etc",
             "Precision production craft & repair",
             "Technicians and related support",
             "Machine operators assmblrs & inspctrs",
             "Transportation and material moving",
             "Executive admin and managerial",
             "Private household services",
             "Armed Forces"],
    "ARACE": ["White",
            "Black",
            "Other",
            "Amer Indian Aleut or Eskimo",
            "Asian or Pacific Islander"],
    "AREORGN": ["Mexican (Mexicano)",
              "Mexican-American",
              "Puerto Rican",
              "Central or South American",
              "All other",
              "Other Spanish",
              "Chicano",
              "Cuban",
              "Do not know",
              "NA"],
    "ASEX": ["Female", "Male"],
    "AUNMEM": ["Not in universe", "No", "Yes"],
    "AUNTYPE": ["Not in universe",
              "Re-entrant",
              "Job loser - on layoff",
              "New entrant",
              "Job leaver",
              "Other job loser"],
    "AWKSTAT": ["Children or Armed Forces",
              "Full-time schedules",
              "Unemployed part- time",
              "Not in labor force",
              "Unemployed full-time",
              "PT for non-econ reasons usually FT",
              "PT for econ reasons usually PT",
              "PT for econ reasons usually FT"],
    "FILESTAT": ["Nonfiler",
               "Joint one under 65 & one 65+",
               "Joint both under 65",
               "Single",
               "Head of household", 
               "Joint both 65+"],
    "GRINREG": ["Not in universe",
              "South",
              "Northeast",
              "West",
              "Midwest", 
              "Abroad"],
    "GRINST": ["Not in universe",
             "Utah",
             "Michigan",
             "North Carolina",
             "North Dakota", 
             "Virginia", 
             "Vermont", 
             "Wyoming", 
             "West Virginia", 
             "Pennsylvania", 
             "Abroad", 
             "Oregon", 
             "California", 
             "Iowa", 
             "Florida", 
             "Arkansas", 
             "Texas", 
             "South Carolina", 
             "Arizona", 
             "Indiana", 
             "Tennessee", 
             "Maine", 
             "Alaska", 
             "Ohio", 
             "Montana", 
             "Nebraska", 
             "Mississippi", 
             "District of Columbia", 
             "Minnesota", 
             "Illinois",
             "Kentucky", 
             "Delaware", 
             "Colorado", 
             "Maryland", 
             "Wisconsin", 
             "New Hampshire", 
             "Nevada", 
             "New York", 
             "Georgia", 
             "Oklahoma", 
             "New Mexico", 
             "South Dakota", 
             "Missouri", 
             "Kansas", 
             "Connecticut", 
             "Louisiana", 
             "Alabama", 
             "Massachusetts", 
             "Idaho", 
             "New Jersey"],
    "HHDFMX": ["Child <18 never marr not in subfamily", 
             "Other Rel <18 never marr child of subfamily RP", 
             "Other Rel <18 never marr not in subfamily", 
             "Grandchild <18 never marr child of subfamily RP", 
             "Grandchild <18 never marr not in subfamily", 
             "Secondary individual", 
             "In group quarters", 
             "Child under 18 of RP of unrel subfamily", 
             "RP of unrelated subfamily", 
             "Spouse of householder", 
             "Householder", 
             "Other Rel <18 never married RP of subfamily", 
             "Grandchild <18 never marr RP of subfamily", 
             "Child <18 never marr RP of subfamily", 
             "Child <18 ever marr not in subfamily", 
             "Other Rel <18 ever marr RP of subfamily", 
             "Child <18 ever marr RP of subfamily", 
             "Nonfamily householder", 
             "Child <18 spouse of subfamily RP", 
             "Other Rel <18 spouse of subfamily RP", 
             "Other Rel <18 ever marr not in subfamily", 
             "Grandchild <18 ever marr not in subfamily", 
             "Child 18+ never marr Not in a subfamily", 
             "Grandchild 18+ never marr not in subfamily", 
             "Child 18+ ever marr RP of subfamily", 
             "Other Rel 18+ never marr not in subfamily", 
             "Child 18+ never marr RP of subfamily", 
             "Other Rel 18+ ever marr RP of subfamily", 
             "Other Rel 18+ never marr RP of subfamily", 
             "Other Rel 18+ spouse of subfamily RP", 
             "Other Rel 18+ ever marr not in subfamily", 
             "Child 18+ ever marr Not in a subfamily", 
             "Grandchild 18+ ever marr not in subfamily", 
             "Child 18+ spouse of subfamily RP", 
             "Spouse of RP of unrelated subfamily", 
             "Grandchild 18+ ever marr RP of subfamily", 
             "Grandchild 18+ never marr RP of subfamily", 
             "Grandchild 18+ spouse of subfamily RP"],
    "HHDREL": ["Child under 18 never married", 
             "Other relative of householder", 
             "Nonrelative of householder", 
             "Spouse of householder", 
             "Householder", 
             "Child under 18 ever married", 
             "Group Quarters- Secondary individual", 
             "Child 18 or older"],
    "MIGMTR1": ["Not in universe", 
              "Nonmover", 
              "MSA to MSA", 
              "NonMSA to nonMSA", 
              "MSA to nonMSA", 
              "NonMSA to MSA", 
              "Abroad to MSA", 
              "Not identifiable", 
              "Abroad to nonMSA"],
    "MIGMTR3": ["Not in universe", 
              "Nonmover", 
              "Same county",
              "Different county same state",
              "Different state same division",
              "Abroad",
              "Different region",
              "Different division same region"],
    "MIGMTR4": ["Not in universe",
              "Nonmover",
              "Same county",
              "Different county same state",
              "Different state in West",
              "Abroad",
              "Different state in Midwest",
              "Different state in South",
              "Different state in Northeast"],
    "MIGSAME": ["Not in universe under 1 year old", "Yes", "No"],
    "MIGSUN": ["Not in universe", "Yes", "No"],
    "PARENT": ["Both parents present",
             "Neither parent present",
             "Mother only present",
             "Father only present",
             "Not in universe"],
    "PEFNTVTY": ["India",
               "Mexico",
               "United-States",
               "Puerto-Rico",
               "Dominican-Republic",
               "England", 
               "Honduras", 
               "Peru", 
               "Guatemala", 
               "Columbia", 
               "El-Salvador", 
               "Philippines", 
               "France", 
               "Ecuador", 
               "Nicaragua", 
               "Cuba", 
               "Outlying-U S (Guam USVI etc)", 
               "Jamaica", 
               "South Korea", 
               "China", 
               "Germany", 
               "Yugoslavia", 
               "Canada", 
               "Vietnam", 
               "Japan", 
               "Cambodia", 
               "Ireland", 
               "Laos", 
               "Haiti", 
               "Portugal", 
               "Taiwan", 
               "Holand-Netherlands", 
               "Greece", 
               "Italy", 
               "Poland", 
               "Thailand", 
               "Trinadad&Tobago", 
               "Hungary", 
               "Panama", 
               "Hong Kong", 
               "Scotland", 
               "Iran"],
    "PEMNTVTY": ["United-States", 
               "Mexico", 
               "Puerto-Rico", 
               "Peru", 
               "Canada", 
               "South Korea", 
               "India", 
               "Japan", 
               "Haiti", 
               "El-Salvador", 
               "Dominican-Republic", 
               "Portugal", 
               "Columbia", 
               "England", 
               "Thailand", 
               "Cuba", 
               "Laos", 
               "Panama", 
               "China", 
               "Germany", 
               "Vietnam", 
               "Italy", 
               "Honduras", 
               "Outlying-U S (Guam USVI etc)", 
               "Hungary", 
               "Philippines", 
               "Poland", 
               "Ecuador", 
               "Iran", 
               "Guatemala", 
               "Holand-Netherlands", 
               "Taiwan", 
               "Nicaragua", 
               "France", 
               "Jamaica", 
               "Scotland", 
               "Yugoslavia", 
               "Hong Kong", 
               "Trinadad&Tobago", 
               "Greece", 
               "Cambodia", 
               "Ireland"],
    "PENATVTY": ["United-States",
               "Mexico", 
               "Puerto-Rico", 
               "Peru", 
               "Canada", 
               "South Korea", 
               "India", 
               "Japan", 
               "Haiti", 
               "El-Salvador", 
               "Dominican-Republic", 
               "Portugal", 
               "Columbia", 
               "England", 
               "Thailand", 
               "Cuba", 
               "Laos", 
               "Panama", 
               "China", 
               "Germany",
               "Vietnam", 
               "Italy", 
               "Honduras", 
               "Outlying-U S (Guam USVI etc)", 
               "Hungary", 
               "Philippines", 
               "Poland", 
               "Ecuador", 
               "Iran", 
               "Guatemala", 
               "Holand-Netherlands", 
               "Taiwan", 
               "Nicaragua", 
               "France", 
               "Jamaica", 
               "Scotland", 
               "Yugoslavia", 
               "Hong Kong", 
               "Trinadad&Tobago", 
               "Greece", 
               "Cambodia", 
               "Ireland"],
    "PRCITSHP": ["Native- Born in the United States", 
               "Foreign born- Not a citizen of U S", 
               "Native- Born in Puerto Rico or U S Outlying", 
               "Native- Born abroad of American Parent(s)", 
               "Foreign born- U S citizen by naturalization"],
    "VETQVA": ["Not in universe", "Yes", "No"],
    "target": ["- 50000.", "50000+."]
}

In [11]:
# Extract unique values
categorical_cols = train_data.select_dtypes(include=['object']).columns

In [12]:
for column in categorical_cols:
    unique_values = list(train_data[column].unique())
    expected_values = EXPECTED_UNIQUE_VALUES[column]

    unmatch_values = list(set(unique_values).difference(set(expected_values)))

    if (len(unmatch_values) > 0): 
        print("Column %s has unmatched values: %s" % (column, ", ".join(unmatch_values)))

Column GRINST has unmatched values: ?
Column MIGMTR1 has unmatched values: ?
Column MIGMTR3 has unmatched values: ?
Column MIGMTR4 has unmatched values: ?
Column MIGSUN has unmatched values: ?
Column PEFNTVTY has unmatched values: ?
Column PEMNTVTY has unmatched values: ?
Column PENATVTY has unmatched values: ?


In [14]:
for column in categorical_cols:
    unique_values = list(test_data[column].unique())
    expected_values = EXPECTED_UNIQUE_VALUES[column]

    unmatch_values = list(set(unique_values).difference(set(expected_values)))

    if (len(unmatch_values) > 0): 
        print("Column %s has unmatched values: %s" % (column, ", ".join(unmatch_values)))

Column GRINST has unmatched values: ?
Column MIGMTR1 has unmatched values: ?
Column MIGMTR3 has unmatched values: ?
Column MIGMTR4 has unmatched values: ?
Column MIGSUN has unmatched values: ?
Column PEFNTVTY has unmatched values: ?
Column PEMNTVTY has unmatched values: ?
Column PENATVTY has unmatched values: ?


Note: Knowing for unknown values, they will be represented by `?`. I am not changing it to anything else.

In [15]:
# Cast ADTIND (Industry code to categorical)
train_data["ADTIND"] = train_data["ADTIND"].apply(str)
test_data["ADTIND"] = test_data["ADTIND"].apply(str)

# Export processed csvs

In [16]:
train_data.to_csv("./data/census_income_learn_processed.csv", index=False)

In [17]:
test_data.to_csv("./data/census_income_test_processed.csv", index=False)